<a href="https://colab.research.google.com/github/ManelSoengas/NLP_Curs/blob/main/Utilitzant_Transformadors_Capacitats_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Capacitats dels tokenizer ràpids**

---

Els tokenitzadors, especialment els recolzats per la biblioteca Tokenizers, poden fer molt més.
Els tokenizers lents són els escrits en Python dins de la biblioteca **Transformers**, mentre que les versions ràpides són les que ofereix **Tokenizers**, que estan escrites en Rust.

A més de les seves capacitats de paral·lelització, la funcionalitat clau dels tokenitzadors ràpids és que sempre fan un seguiment de l'abast original de textos d'on provenen els testimonis finals, una característica que anomenem mapatge compensat . Això, al seu torn, desbloqueja funcions com l'assignació de cada paraula a les fitxes que ha generat o la correspondència de cada caràcter del text original al testimoni que hi ha dins, i viceversa.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
encoding = tokenizer(example)
print(type(encoding))

In [ ]:
tokenizer.is_fast

In [ ]:
encoding.is_fast

In [ ]:
#encoding.tokens()
print(" ".join(encoding.tokens()))


In [ ]:
encoding.word_ids()

Es pot observar que les fitxes especials del tokenizer [CLS]i [SEP]estan assignades a None, i després cada testimoni s'assigna a la paraula de la qual prové. Això és especialment útil per determinar si un testimoni està a l'inici d'una paraula o si dos testimonis estan a la mateixa paraula. Podríem confiar en el ##prefix per a això, però només funciona per a tokenitzadors semblants a BERT; aquest mètode funciona per a qualsevol tipus de tokenitzador sempre que sigui ràpid. En el següent capítol, veurem com podem utilitzar aquesta capacitat per aplicar correctament les etiquetes que tenim per a cada paraula als testimonis en tasques com el reconeixement d'entitats amb nom (NER) i l'etiquetatge de part de la veu (POS). També el podem utilitzar per emmascarar totes les fitxes procedents de la mateixa paraula en el modelatge de llenguatge emmascarat (una tècnica anomenada emmascarament de paraules completes ).

In [ ]:
start, end = encoding.word_to_chars(3)
example[start:end]

Finalment, podem assignar qualsevol paraula o testimoni als caràcters del text original, i viceversa, mitjançant els mètodes word_to_chars()o token_to_chars()i char_to_word()o . char_to_token()Per exemple, el word_ids()mètode ens va dir que ##ylforma part de la paraula de l'índex 3, però quina paraula és a la frase? Ho podem esbrinar així:

# **Pipeline de classificació**